In [2]:
# Paso 1: Cargar las librerías necesarias

import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

c:\Users\yera_\Documents\Areas\En proceso\Qualentum-Proyectos\ProspectAI\ProspectAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Determinar el dispositivo a usar (CPU o GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [9]:
img_path = "../prospectos/train/alprazolan/alprazolan.JPG"
# Abre la imagen directamente desde la ruta
image = Image.open(img_path)
prompt = ('Examine the medical leaflet in the image and list the most important details related to: '
'- What the medicine is.' 
'- What the medicine is used for.'
'- Precautions before taking it.'
' - Way to take it.'
'- Adverse effects of the drug. '
'- Conservation of the drug.'
'- Additional Information. '
'Finally, translate into Spanish.')

In [5]:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["mm_projector", "vision_model"],
)

In [6]:
model_id = "qresearch/llama-3-vision-alpha-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    quantization_config=bnb_cfg,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:42<00:00, 10.65s/it]


In [10]:
print(
    tokenizer.decode(
        model.answer_question(image, prompt, tokenizer),
        skip_special_tokens=True,
    )
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here is the translation of the medical leaflet into Spanish:

**¿Qué es este medicamento?**

Este medicamento es un [insert type of medication, e.g. antibiótico, antihistamínico, etc.] utilizado para tratar [insert condition or disease].

**¿Para qué se utiliza este medicamento?**

Se utiliza para tratar [insert condition or disease] y prevenir [insert complication or secondary condition].

**¿Qué precauciones debo tomar antes de tomar este medicamento?**

* [Insert precaution 1, e.g. inform your doctor if you have a history of allergies]
* [Insert precaution 2, e.g. avoid taking the medication with certain foods]
* [Insert precaution 3, e.g. inform your doctor if you are pregnant or breastfeeding]

**¿Cómo debo tomar este medicamento?**

* [Insert dosage instructions, e.g. take 1 tablet twice a day]
* [Insert administration instructions, e.g. swallow the tablet whole]
* [Insert duration of treatment, e.g. take the medication for 7 days]

**¿Qué efectos secundarios pueden ocurrir como 